In [1]:
import gensim
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# df=pd.read_json("../data/Cell_Phones_and_Accessories_5.json",lines=True)

In [3]:
df=pd.read_csv("../data/translated/kurdish-reviews.csv")
# df=pd.read_csv("../data/medical-kurdish-dataset.csv")

In [6]:
df.KurdishText


0       چەندین بەرهەمی خۆراکی سەگی قوتووکراوی ڤیتالیتی...
1       بەرهەمەکە گەیشتە لای کە بە ناوی Jumbo Salted P...
2       ئەمە شیرینییەکە کە لە دەوروبەری چەند سەدەیەکدا...
3       ئەگەر بەدوای پێکهاتەی نهێنی ڕۆبیتوسیندا دەگەڕێ...
4       تافی نایاب بە نرخێکی نایاب. جۆرێکی بەرفراوانی ...
                              ...                        
1995    دەبێت پێبکەنم بەو پێداچوونەوەیانەی کە دەیانگوت...
1996    ماوەیەکی زۆر لەمەوبەر چەند پێداچوونەوەیەکی لەب...
1997    چاوەڕوانی شتی گەورە بووم بە پشتبەستن بە پێداچو...
1998    من ئەم تێکەڵەی پانکەیکەم زۆر خۆشدەوێت. نزیکەی ...
1999    ئەتوانم چی بڵێم?? ئەوان نایاب و سەرنجڕاکێشن، و...
Name: KurdishText, Length: 2000, dtype: object

In [7]:

def preprocess_text(text):
    try:
        return gensim.utils.simple_preprocess(text)
    except TypeError:
        print("typerror for text: ",text)
        return []


In [8]:
df['PreprocessedText']=df.KurdishText.apply(preprocess_text)

typerror for text:  nan
typerror for text:  nan
typerror for text:  nan


In [9]:
df.head()

,Id,KurdishText,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,PreprocessedText
0,1,چەندین بەرهەمی خۆراکی سەگی قوتووکراوی ڤیتالیتی...,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,"[چەندین, بەرهەمی, خۆراکی, سەگی, قوتووکراوی, ڤی..."
1,2,بەرهەمەکە گەیشتە لای کە بە ناوی Jumbo Salted P...,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,"[بەرهەمەکە, گەیشتە, لای, کە, بە, ناوی, jumbo, ..."
2,3,ئەمە شیرینییەکە کە لە دەوروبەری چەند سەدەیەکدا...,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,"[ئەمە, شیرینییەکە, کە, لە, دەوروبەری, چەند, سە..."
3,4,ئەگەر بەدوای پێکهاتەی نهێنی ڕۆبیتوسیندا دەگەڕێ...,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,"[ئەگەر, بەدوای, پێکهاتەی, نهێنی, ڕۆبیتوسیندا, ..."
4,5,تافی نایاب بە نرخێکی نایاب. جۆرێکی بەرفراوانی ...,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,"[تافی, نایاب, بە, نرخێکی, نایاب, جۆرێکی, بەرفر..."


In [14]:

res={}
df['KurdishText'] = df['KurdishText'].fillna('')
for i,row in tqdm(df.iterrows(),total=len(df)):
    try:
        text=row.KurdishText
        myId=row.Id
        gensim_results=gensim.utils.simple_preprocess(text)
        res[myId]=gensim_results
    except TypeError:
        print("typerror for text: ",text)
        continue



  0%|          | 0/2000 [00:00<?, ?it/s]

In [17]:
# continuing with old way not looping
review_text=df.KurdishText.apply(gensim.utils.simple_preprocess)

In [18]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [19]:
model.build_vocab(review_text,progress_per=100)

In [20]:
model.epochs

5

In [21]:
model.corpus_count

2000

In [22]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(455958, 603660)

In [23]:
model.save("../models/kurdish-word2vec-2000.model")

In [24]:
model.wv.most_similar("با")

[('com', 0.9997527003288269),
 ('gp', 0.9997270703315735),
 ('amazon', 0.9996203780174255),
 ('www', 0.9993019104003906),
 ('http', 0.9992315769195557),
 ('href', 0.9990754723548889),
 ('چیپس', 0.9981304407119751),
 ('کتری', 0.9979948997497559),
 ('دیجۆن', 0.9979345798492432),
 ('پاکێجی', 0.9976974129676819)]

In [ ]:
model.wv.similarity(w1="expensive",w2="costly")

0.6865411